In [ ]:
#| default_exp machine_learning.llm_notation_summarization

# machine_learning.llm_notation_summarization
> Functions for summarizing notations using a large language model.

Previously, `trouver` used fine-tuned Transformers based models (Specifically Google's T5) to summarize notation. However, procuring high quality data is highly time consuming for this approach and the fine-tuned models have yet produce generally competent predictions.

This module instead opts to use large language models (LLM's)/generative AI models for the summarization task.

In [ ]:
#| export
from os import PathLike
from typing import Optional, TypedDict
import re

import lmstudio as lms
from lmstudio import LLM

from trouver.obsidian.file import MarkdownFile, MarkdownLineEnum

from trouver.machine_learning.notation_summarization import _summary_should_be_generated, _get_summary, get_latex_in_original_from_parsed_notation_note_data, single_input_for_notation_summarization, notation_summarization_data_from_note, _write_summary_to_notation_note, format_training_tokens, format_for_gemma_instruct


from trouver.personal_vault.note_processing import process_standard_information_note
from trouver.notation.parse import main_of_notation, parse_notation_note
from trouver.obsidian.vault import VaultNote

from trouver.obsidian.links import links_from_text

In [ ]:
from trouver.helper.tests import _test_directory

In [ ]:
#| export

# --- 1. SYSTEM PROMPT (TARGET + TEXT SPLIT) ---
NOTATION_SUMMARIZATION_SYSTEM_PROMPT = r"""
# Role
You are a strict, non-conversational mathematical compiler. Your task is to extract a rigorous definition summary from the provided text for a specific **Target Symbol**.

# Input Data
1.  **Text:** The raw mathematical excerpt containing the definition.
2.  **Target:** The specific notation to extract and define from the text above.

# Goal
Generate the prose continuation of a sentence starting with: "$SYMBOL$ [[SOURCE|denotes]]..."
**Start your output immediately with the noun phrase.** The output must be mathematically precise, grammatically correct prose where all mathematics in valid MathJax/LaTeX that is renderable in Obsidian.md markdown.

# Generation Rules

## 1. The "Identity Signature"
- **Priority:** Use the formal name if given (e.g., "the norm," "the sheaf of differentials").
- **Parent Context:** Immediately link the symbol to its primary parameters in the first sentence (e.g., "of a morphism $f$," "of an ideal $\mathfrak{a}$").

## 2. The Operational Definition
- Explain **how** it is constructed using the **exact abstraction level** of the text.
- **Justification:** Keep "because" clauses that explain well-definedness (e.g., "finite because it is full-rank").
- **Procedural Logic:** If the text lists steps (1, 2, 3), use a numbered list.
- **Formulas:** Include the defining equation.

## 3. Context Reconstruction ("Where" Clause) 
- **Recursive Unwinding:** Do not just list variables; **briefly define them**.   
- *Bad:* "...where $cl_X$ is the cycle class map."  
- *Good:* "...where $cl_X: CH^i(X) \to H^{2i}(X)$ is the cycle class map."
 - **Integration:** Use a "where..." clause immediately following the definition to define variables like $K, \mathcal{O}_K$, $X, Y$.

## 4. Structural Characterizations
- **New Paragraph:** Start a new paragraph for distinct properties or theorems.
- **Subject Focus:** Only include statements that describe the target symbol itself.
- **Negative Constraint:** Do not include definitions of *other* concepts (like "Catenary Ring") just because they mention the target symbol.

## 5. Constraints
- **Tone:** Clinical, precise. No "We defined," "Note that."
- **Safety:** ABSOLUTELY NO EMOJIS. NO MARKDOWN CODE BLOCKS.

## 6. LaTeX Standardization (MANDATORY)
- **Error Correction:** You must fix syntactic errors in the input. If the text has `\frac a b` (ambiguous) or `\alph` (typo), output correct standard LaTeX: `\frac{a}{b}`, `\alpha`.
- **Obsidian Format:**
  - Use `$` for inline math.
  - Use `$$` for block math/equations.
  - Use `$$\begin{align*} ... \end{align*}$$` for multi-line definitions.
  - NEVER use `\(` `\)` `\[` `\]`.
- **Notation:**
  - Convert plain text operators to commands: "Gal" -> `\operatorname{Gal}`, "Hom" -> `\operatorname{Hom}` (or use \mathrm or \mathbf, etc. as appropriate for the text).

## 7. Input Sanitization

    Text Cleaning: If the input text contains typos (e.g., "morphism off schemes", "defind as"), correct the spelling in your prose output.

    Variable Consistency: If the input text uses inconsistent notation for the same object (e.g., switching between $\epsilon$ and $\varepsilon$ randomly), standardize to the most common or standard usage in the definition.

    Noise Removal: Ignore filler words like "clearly," "obviously," or conversational asides found in the source text.
---

# Example 1: Constructive Object
**Text:**
Let $f: X \to Y$ be a morphism of schemes... The sheaf of relative differentials, denoted $\Omega_{X/Y}$, is then defined to be the pullback of this conormal sheaf... That is, $\Omega_{X/Y} := \Delta^*(\mathcal{I}/\mathcal{I}^2)$.
**Target:** \Omega_{X/Y}
**Output:**
the sheaf of relative differentials of a morphism $f: X \to Y$ of schemes. It is defined as the pullback $\Omega_{X/Y} := \Delta^*(\mathcal{I}/\mathcal{I}^2)$, where $\mathcal{I}$ is the ideal sheaf of the diagonal immersion $\Delta: X \to X \times_Y X$. Since the conormal sheaf $\mathcal{I}/\mathcal{I}^2$ is a sheaf on the diagonal $\Delta(X)$ (viewed as a sheaf on $X$), the result is a sheaf on $X$.

An important property is that for affine open subschemes $U \subseteq X$ and $V \subseteq Y$ with $f(U) \subseteq V$, the sections $\Omega_{X/Y}(U)$ are isomorphic to the module of relative differentials $\Omega_{A/R}$.

# Example 2: Property/Value
**Text:**
Let $K$ be a number field... We define the norm of an ideal $\mathfrak{a}$, denoted $N(\mathfrak{a})$, to be the cardinality of the quotient ring $\mathcal{O}_K / \mathfrak{a}$. Since $\mathfrak{a}$ is a full-rank sublattice of $\mathcal{O}_K$, this quotient ring is always finite.
**Target:** N(\mathfrak{a})
**Output:**
the norm of a non-zero ideal $\mathfrak{a}$ in the ring of integers $\mathcal{O}_K$ of a number field $K$. It is defined as the cardinality of the quotient ring $N(\mathfrak{a}) := |\mathcal{O}_K / \mathfrak{a}|$. This quotient is always finite because $\mathfrak{a}$ is a full-rank sublattice of $\mathcal{O}_K$.

The norm is a completely multiplicative function, meaning $N(\mathfrak{a}\mathfrak{b}) = N(\mathfrak{a})N(\mathfrak{b})$ for any non-zero ideals $\mathfrak{a}, \mathfrak{b}$.

# Example 3: Procedural Definition
**Text:**
Thus if $D \in \operatorname{Div}(V)$ is a divisor and we want to choose a particular height function $h_{V, D}$, we need to make the following choices:
[1] Choose very ample divisors $D_{1}$ and $D_{2}$ with $D=D_{1}-D_{2}$.
...
**Target:** h_{V,D}
**Output:**
a particular height function for a divisor $D$ on a variety $V$. It is defined by the following construction:
1. Choose very ample divisors $D_{1}$ and $D_{2}$ such that $D=D_{1}-D_{2}$.
2. Choose embeddings $\phi_{1}: V \rightarrow \mathbb{P}^{n}$ and $\phi_{2}: V \rightarrow \mathbb{P}^{m}$ corresponding respectively to $D_{1}$ and $D_{2}$.
3. Set $h_{V, D}(P)=h\left(\phi_{1}(P)\right)-h\left(\phi_{2}(P)\right)$.

# Example 4: Inconsistency Standardization
**Text:**
The set of morphisms between schemes X and Y is usually writen Hom(X,Y). We say that a morphism $f \in \operatorname{Hom}(X, Y)$ is separated if the diagonal is a closed immersion.
**Target:** \operatorname{Hom}(X,Y)
**Output:**
the set of morphisms between schemes $X$ and $Y$. 

"""



In [ ]:
#| export
def example_from_notation_note(
        notation_note: VaultNote
        ) -> dict[str, str]: # Keys include 'processed_main_note_content' and 'latex_in_original'
    """
    Helper to parse `notation_note` and gather data for summarization.
    
    This function extracts the LaTeX used in the original main note and 
    retrieves the processed content of that main note.
    """
    parsed = parse_notation_note(notation_note)

    # Access the main note (the one where the notation is defined)
    main_note = VaultNote(
        vault=notation_note.vault, name=parsed.name_of_main_note)
    
    # Process the main note's content to remove metadata/formatting
    main_note_mf = MarkdownFile.from_vault_note(main_note)
    main_note_content = str(process_standard_information_note(main_note_mf, notation_note.vault))

    # Identify the specific LaTeX string as it appeared in the source
    latex_in_original = get_latex_in_original_from_parsed_notation_note_data(
            parsed.yaml_frontmatter_meta, parsed.notation_str)
    
    # Extract existing summary data
    summary_data = notation_summarization_data_from_note(
        notation_note, notation_note.vault, check_for_actual_summarization=False)
    
    return summary_data

In [ ]:
# Setup a path to your test vault
test_vault_path = _test_directory() / 'test_vault_4'
# Example notation note: 'number_theory_reference_1_notation_Z_nZ_ring_of_integers_modulo_n'
example_note = VaultNote(test_vault_path, name='number_theory_reference_1_notation_Z_nZ_ring_of_integers_modulo_n')

# Run the helper
data = example_from_notation_note(example_note)

# print(data.keys)
# print(f"Notation: {data['notation']}")
print(f"Main Note: {data['main_note_name']}")
print(f"Content: {data['processed_main_note_content']}")
print(f"latex_in_original: {data['latex_in_original']}")
# Output would show the extracted dictionary data

Main Note: number_theory_reference_1_Definition 1.7
Content: The ring of integers modulo $n$, denoted $\mathbb{Z}/n\mathbb{Z}$ has the elements ...

latex_in_original: $\mathbb{Z}/n\mathbb{Z}$


In [ ]:
#| hide
from fastcore.test import test_eq, test_is

# 1. Test basic functionality with a known note
test_vault = _test_directory() / 'test_vault_4'
note_name = r'number_theory_reference_1_notation_Z_nZ_ring_of_integers_modulo_n'
notat_note = VaultNote(test_vault, name=note_name)

res = example_from_notation_note(notat_note)

# Verify expected keys in return dict
expected_keys = {"notation_note_name", "notation", "latex_in_original", "summary", "main_note_name"}
assert expected_keys.issubset(res.keys())

# Verify specific values based on the vault content
test_eq(res['main_note_name'], 'number_theory_reference_1_Definition 1.7')
test_eq(res['notation'], r'$\mathbb{Z}/n\mathbb{Z}$')

# 2. Test behavior with different check_for_actual_summarization logic
# If the note isn't summarized yet, ensure the helper still returns data when False
res_no_check = example_from_notation_note(notat_note)
assert res_no_check is not None

In [ ]:
#| export

#| export
# def notation_note_should_be_summarized(
#         notation_note: VaultNote,
#         vault: PathLike,
#         main_note: Optional[VaultNote] = None, # The main note from which the notation comes from. If this is `None`, then the `main_note` is obtained via the `main_of_notation` function.
#         overwrite_previous_autogenerated_summary: bool = False, # If `True`, overwrite previously autogenerated summaries
#         latex_in_original_comes_first: bool = True # This is a parameter to pass to calls to the `single_input_for_notation_summarization` function. If `True`, the `latex_in_original` piece appears before the `main_note_content`. While the default value of `True` is recommended, passing `False` to this parameter may be necessary to use the older version of the summarization model in the repo [`notation_summarizations_model`](https://huggingface.co/hyunjongkimmath/notation_summarizations_model).
#     ) -> bool:
#     r"""
#     Return `True` if notation note has no summary or an autogenerated summary.

#     Helper to tell
#     """
#     metadata, notation_str, main_note_name,\
#         notation_note_content_mf, _\
#         = parse_notation_note(notation_note, vault)
#     summary_should_be_generated, main_mf = _summary_should_be_generated(
#         main_note, main_note_name, vault, notation_note,
#         overwrite_previous_autogenerated_summary,
#         metadata, notation_note_content_mf)
#     return summary_should_be_generated

def notation_note_should_be_summarized(
        notation_note: VaultNote,
        vault: PathLike,
        main_note: Optional[VaultNote] = None, # The main note from which the notation comes from. If this is `None`, then the `main_note` is obtained via the `main_of_notation` function.
        overwrite_previous_autogenerated_summary: bool = False, # If `True`, overwrite previously autogenerated summaries
        latex_in_original_comes_first: bool = True # This is a parameter to pass to calls to the `single_input_for_notation_summarization` function. If `True`, the `latex_in_original` piece appears before the `main_note_content`. While the default value of `True` is recommended, passing `False` to this parameter may be necessary to use the older version of the summarization model in the repo [`notation_summarizations_model`](https://huggingface.co/hyunjongkimmath/notation_summarizations_model).
    ) -> bool:
    r"""
    Return `True` if a notation note has no summary or contains an autogenerated summary that should be updated.

    This helper determines if the note is a candidate for the summarization pipeline based on:
    1. The presence of the `_auto/notation_summary` tag (if `overwrite_previous_autogenerated_summary` is True).
    2. Whether the note currently lacks content beyond the "denotes" link.
    3. Whether the main reference note actually exists and contains content.
    """
    # parse_notation_note retrieves metadata, the notation string, and the main note's name
    metadata, notation_str, main_note_name,\
        notation_note_content_mf, _\
        = parse_notation_note(notation_note, vault)
        
    # _summary_should_be_generated performs the logic of checking if the main note exists
    # and if the notation note is already "sufficiently summarized" or tagged as auto-generated.
    summary_should_be_generated, _ = _summary_should_be_generated(
        main_note, main_note_name, vault, notation_note,
        overwrite_previous_autogenerated_summary,
        metadata, notation_note_content_mf)
        
    return summary_should_be_generated

In [ ]:
# Setup test environment
test_vault = _test_directory() / 'test_vault_4'
# A note that has already been summarized manually
manual_note = VaultNote(test_vault, name='number_theory_reference_1_notation_Z_nZ_ring_of_integers_modulo_n')

# Check if it needs summarization (it shouldn't, as it has manual content)
needs_summary = notation_note_should_be_summarized(manual_note, test_vault)
print(f"Needs summary: {needs_summary}") 

# A note that only has the "denotes" link and no actual explanation
# empty_note = VaultNote(test_vault, name='some_new_unsummarized_notation')
# print(f"Empty note needs summary: {notation_note_should_be_summarized(empty_note, test_vault)}")

Needs summary: False


In [ ]:
#| hide
from fastcore.test import test_eq
import tempfile
import shutil
from pathlib import Path

# 1. Test with existing summarized note
test_vault = _test_directory() / 'test_vault_4'
summarized_note = VaultNote(test_vault, name='number_theory_reference_1_notation_Z_nZ_ring_of_integers_modulo_n')

# Should be False because it's already summarized and not tagged as _auto
test_eq(notation_note_should_be_summarized(summarized_note, test_vault), False)

# 2. Test with an auto-generated note
with tempfile.TemporaryDirectory() as temp_dir:
    temp_path = Path(temp_dir)
    shutil.copytree(test_vault, temp_path / 'vault')
    v = temp_path / 'vault'
    
    # Simulate an auto-generated note by adding the tag
    auto_note = VaultNote(v, name='number_theory_reference_1_notation_Z_nZ_ring_of_integers_modulo_n')
    mf = MarkdownFile.from_vault_note(auto_note)
    mf.add_tags(['_auto/notation_summary'])
    mf.write(auto_note)
    
    # By default, it shouldn't overwrite
    test_eq(notation_note_should_be_summarized(auto_note, v, overwrite_previous_autogenerated_summary=False), False)
    
    # With overwrite=True, it should return True
    test_eq(notation_note_should_be_summarized(auto_note, v, overwrite_previous_autogenerated_summary=True), True)

# 3. Test behavior when main note is missing
# (Assuming 'non_existent_main' is referenced in a notation note)
missing_main_note = VaultNote(test_vault, name='notation_with_missing_main_reference')
if missing_main_note.exists():
    test_eq(notation_note_should_be_summarized(missing_main_note, test_vault), False)

C:\Users\hyunj\Documents\Development\Python\trouver\trouver\helper\html.py:104: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  parsed_soup = BeautifulSoup(text, 'html.parser')
C:\Users\hyunj\Documents\Development\Python\trouver\trouver\machine_learning\notation_summarization.py:562: UserWarning: The notation note already has contents, so no new summary was added. Notation note name: number_theory_reference_1_notation_Z_nZ_ring_of_integers_modulo_n
  warnings.warn(


In [ ]:
#| export
def separate_thoughts(raw_content: str):
    """
    Separates model 'reasoning' from the actual answer.
    Handles <think>, <thought>, and [THOUGHT] tags.
    """
    # 1. Define common patterns for thinking blocks
    # This regex looks for <think>...</think> or  (case insensitive)
    tag_pattern = r"<(think|thought)>([\s\S]*?)<\/\1>"
    
    match = re.search(tag_pattern, raw_content, re.IGNORECASE)
    
    if match:
        thoughts = match.group(2).strip()
        # Remove the thinking block from the main content
        answer = re.sub(tag_pattern, "", raw_content, flags=re.IGNORECASE).strip()
        return thoughts, answer
    
    # 2. Fallback for models that don't use tags but use a header
    if "THOUGHTS:" in raw_content.upper():
        parts = re.split(r"THOUGHTS:", raw_content, flags=re.IGNORECASE)
        # Assuming format: THOUGHTS: [logic] ANSWER: [result]
        if "ANSWER:" in parts[1].upper():
            sub_parts = re.split(r"ANSWER:", parts[1], flags=re.IGNORECASE)
            return sub_parts[0].strip(), sub_parts[1].strip()
            
    # 3. If no markers found, return everything as the answer
    return None, raw_content.strip()

In [ ]:
ex1 = """First, analyze the problem.<think>Step 1: Identify key components. Step 2: Validate inputs.</think> The final answer is 42."""

thoughts1, answer1 = separate_thoughts(ex1)
test_eq(thoughts1, "Step 1: Identify key components. Step 2: Validate inputs.")
test_eq(answer1, "First, analyze the problem. The final answer is 42.")

In [ ]:
ex2 = """<THOUGHT>
Mathematical reasoning: check base cases first, then induction step.
Edge case x=0 gives y=1.
</THOUGHT>
Final computation: ∫[0,1] x² dx = 1/3"""

thoughts2, answer2 = separate_thoughts(ex2)
test_eq(thoughts2, "Mathematical reasoning: check base cases first, then induction step.\nEdge case x=0 gives y=1.")
test_eq(answer2, "Final computation: ∫[0,1] x² dx = 1/3")


In [ ]:
ex3 = """THOUGHTS: Gal(L/K) is defined via étale cohomology. Verify separability first.
ANSWER: The Galois group Gal(L/K) is finite of order [L:K]."""

thoughts3, answer3 = separate_thoughts(ex3)
test_eq(thoughts3, "Gal(L/K) is defined via étale cohomology. Verify separability first.")
test_eq(answer3, "The Galois group Gal(L/K) is finite of order [L:K].")


In [ ]:
#| hide
from fastcore.test import *

# Tag extraction works
test_eq(separate_thoughts("<think>reasoning</think>")[0], "reasoning")
test_eq(separate_thoughts("<THOUGHT>test</thought>")[0], "test")

# Answer extraction - EXACT original behavior
test_eq(separate_thoughts("A<think>B</think>C")[1], "AC")
test_eq(separate_thoughts("A <think>B</think> C")[1], "A  C")
test_eq(separate_thoughts("A\n<think>B</think>\nC")[1], "A\n\nC")

# Multiple tags - re.sub removes FIRST match + .strip()
multi = "<think>first</think>text<think>second</think>"
test_eq(separate_thoughts(multi)[0], "first")
test_eq(separate_thoughts(multi)[1], "text")  # ← Fixed: .strip() removes trailing tag

# No tags
test_eq(separate_thoughts("plain text")[0], None)
test_eq(separate_thoughts("plain text")[1], "plain text")

# Malformed tags
test_eq(separate_thoughts("<think>no close")[0], None)
test_eq(separate_thoughts("<think>no close")[1], "<think>no close")

# THOUGHTS:ANSWER: fallback
test_eq(separate_thoughts("THOUGHTS: abc ANSWER: def"), ("abc", "def"))

# Empty cases
test_eq(separate_thoughts("<think></think>")[0], "")
test_eq(separate_thoughts("")[1], "")
test_eq(separate_thoughts("   ")[1], "")


In [ ]:
#| export
# --- 2. CORE FUNCTIONS ---

class SummaryResponse(TypedDict):
    thoughts: str
    output: str

def generate_notation_summary(
        model: LLM,
        excerpt_text: str,
        target_symbol: str,
        max_context: int = 4096,
        verbose: bool = True,
        return_thoughts: bool = False,
        system_prompt: str = NOTATION_SUMMARIZATION_SYSTEM_PROMPT,
        ) -> str | SummaryResponse:
    # 1. PRE-CALCULATE TOKENS FOR TRUNCATION
    # Use Text -> Target order to favor caching
    sys_tokens = len(model.tokenize(NOTATION_SUMMARIZATION_SYSTEM_PROMPT))
    target_overhead = len(model.tokenize(f"\n\nTarget Symbol: {target_symbol}\nOutput:"))
    safety_margin = 500 
    
    available_for_text = max_context - sys_tokens - safety_margin - target_overhead
    # excerpt_tokens = model.tokenize(excerpt_text)


    # 2. SMART TRUNCATION (Using .decode instead of .detokenize)
    # 2. SMART TRUNCATION
    estimated_char_limit = available_for_text * 3
    
    if len(excerpt_text) > estimated_char_limit:
        # Slice the string directly
        excerpt_text = excerpt_text[:estimated_char_limit]
        
        if verbose:
            print(f"Warning: Excerpt truncated to {len(excerpt_text)} characters (approx. {available_for_text} tokens).")


    # 3. DEFINE STRUCTURED MESSAGES (Text first for Caching)
    # Keeping the System Prompt as a separate object allows LM Studio to cache it.
    # The 'Text' is now first to ensure it's part of the stable prefix, which basically means that the model's cache will be able to reload up to the excerpt_text before consider each target_symbol.
    user_content = f"Text:\n{excerpt_text}\n\nTarget Symbol: {target_symbol}\nOutput:"
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_content}
    ]
    # 2. BUILD FLAT PROMPT (system + user)
    # prompt = SYSTEM_PROMPT + "\n\n" + f"Target: {target_symbol}\nText: {excerpt_text}\nOutput:"

    # 3. GENERATE (no .chat, use .respond)
    try:
        result = model.respond({"messages": messages}, config={"temperature": 0.1})
        raw_text = str(result).strip()

        # SEPARATE THOUGHTS FROM ANSWER
        thoughts, clean_answer = separate_thoughts(raw_text)
        
        # You can log thoughts for debugging, but return the clean answer
        if verbose and thoughts:
            print(f"\n[Model Logic Trace]: {thoughts[:100]}...")
        if return_thoughts and thoughts:
            return SummaryResponse(thoughts=thoughts, output=clean_answer)
        else:
            return clean_answer
    except Exception as e:
        print(f"LLM Generation Error: {e}")
        return ""


In [ ]:
# Models on lms may be loaded as follows:

# Load methods
# import lmstudio as lms
# from lmstudio import LLM
# from typing import TypedDict

# Initialize Model; do one of the following:
# model = lms.llm("google/gemma-3-27b") # This loads a model that has been already been downloaded in lmstudio.
# model = lms.llm() # Run this after loading a model on lmstduio.

# target = r"ht(\mathfrak{p})"
# raw_text = r"""Let $A$ be a commutative ring... The height of a prime ideal $\mathfrak{p}$, which we will denote by ht(\mathfrak{p}), is defined to be the Krull dimension of the localization of $A$ at $\mathfrak{p}$, that is, $ht(\mathfrak{p}) := \dim(A_\mathfrak{p})$."""

# print(f"Target: {target}")
# summary = generate_notation_summary(model, raw_text, target, return_thoughts=True)

# if isinstance(summary, str):
#     print("\n--- Model Output ---")
#     print(summary)
#     print("--------------------")
# else:
#     print("\n--- Model Output ---")
#     print(summary["output"])
#     print("--------------------")
#     print("\n--- Model Thoughts ---")
#     print(summary["thoughts"])
#     print("--------------------")


In [ ]:
#| exec
from types import SimpleNamespace
from typing import TypedDict

class SummaryResponse(TypedDict):
    thoughts: str
    output: str

class MockLLM:
    def tokenize(self, text): 
        # Return LIST of token IDs (what real LLMs return)
        return list(range(len(text) // 4 + 1))  # Mock token IDs
    
    def respond(self, messages, config=None):
        return """<think>First identify Gal(L/K) in text. Then summarize its role in context.</think>
Gal(L/K) is the Galois group of the extension L/K, measuring symmetries of the field extension."""

mock_llm = MockLLM()
NOTATION_SUMMARIZATION_SYSTEM_PROMPT = "Summarize the mathematical role of the target symbol in the given text."

# Now works!
summary1 = generate_notation_summary(
    mock_llm, 
    "Let L/K be Galois. Gal(L/K) acts on roots.", 
    "Gal(L/K)"
)
print("Summary1:", repr(summary1))
test_eq(len(mock_llm.tokenize("test")), 2)  # len() works on list



[Model Logic Trace]: First identify Gal(L/K) in text. Then summarize its role in context....
Summary1: 'Gal(L/K) is the Galois group of the extension L/K, measuring symmetries of the field extension.'


In [ ]:
#| exec
# Example 2: Return thoughts
response2 = generate_notation_summary(
    mock_llm, 
    "The étale cohomology computes Gal(L/K).", 
    "Gal(L/K)",
    return_thoughts=True
)
test_eq(response2["thoughts"], "First identify Gal(L/K) in text. Then summarize its role in context.")
test_eq(response2["output"], "Gal(L/K) is the Galois group of the extension L/K, measuring symmetries of the field extension.")



[Model Logic Trace]: First identify Gal(L/K) in text. Then summarize its role in context....


In [ ]:
#| exec
# Example 3: Truncation
long_text = "Mathematical text about Galois groups..." * 200
summary3 = generate_notation_summary(mock_llm, long_text, "Gal(L/K)", verbose=True)
print("Truncated:", len(summary3) > 0)



[Model Logic Trace]: First identify Gal(L/K) in text. Then summarize its role in context....
Truncated: True


In [ ]:
#| hide
from fastcore.test import *

mock_llm_no_thoughts = MockLLM()
mock_llm_no_thoughts.respond = lambda s,m,c: "Direct answer"
mock_llm_no_thoughts.tokenize = lambda s,t: list(range(len(t)//4 + 1))

# Tokenization works
assert len(mock_llm.tokenize("test text")) > 0
assert isinstance(mock_llm.tokenize("test"), list)

# Function works
result = generate_notation_summary(mock_llm, "text", "symbol")
assert isinstance(result, str)
assert len(result) > 0

# Return types
str_result = generate_notation_summary(mock_llm, "text", "symbol", return_thoughts=False)
assert isinstance(str_result, str)

dict_result = generate_notation_summary(mock_llm, "text", "symbol", return_thoughts=True)
assert isinstance(dict_result, dict)
assert "thoughts" in dict_result
assert "output" in dict_result

# Error handling
class FailingLLM:
    def tokenize(self, t): return []
    def respond(self, m, c): raise ValueError("fail")
assert generate_notation_summary(FailingLLM(), "text", "symbol") == ""



[Model Logic Trace]: First identify Gal(L/K) in text. Then summarize its role in context....

[Model Logic Trace]: First identify Gal(L/K) in text. Then summarize its role in context....

[Model Logic Trace]: First identify Gal(L/K) in text. Then summarize its role in context....
LLM Generation Error: FailingLLM.respond() got an unexpected keyword argument 'config'
